# ReAct :- Reasoning and Acting

ReAct is a new approach that combines the reasoning(chain-of-thoughts prompting) and acting capabilities of LLMs.

With ReAct, LLMs generate reasoning traces and task-specific actions in an interleaved manner. This means that the model can reason about the task and then take actions in the real world to gather more information or complete the task. This combination of reasoning and acting allows ReAct to overcome some of the limitations of existing LLMs, such as hallucinations  and error propagation.

**LangChain Agents: Tools + Chains**
LangChain agents are one of the key components that make LangChain so useful. They take LangChain tools and chains, and combine them into one agent. 

## Creating a ReAct Agent
The big problem when using only tools or chains is that you probably won't want to use the same tool with every query you send.
In our previous examples, we used as tools Wikipedia or DuckDuckGo but not both. Using the **ReAct agent**, it will check the input and then decide what is the best tool to use based on the user's query.

**ImportantNote**:-
In the following cell we are installing the **langchainhub package**. The langchainhub is a hub for sharing and discovering resources related to LangChain. It stores and allows access to pre-trained prompts, chains, and agents that can be used in LangChain projects.

In [1]:
pip install langchainhub -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
# importing the modules for prompts, agents, tools, and external services
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

#Prepare the prompt template for asking questions. This template will structure the way the questions are presented to the agent.
template = '''
Answer the following questions in Hindi as best as you can.
Questions: {q}
'''   # a multiline string


prompt_template = PromptTemplate.from_template(template)

#Retrieve the structure of the ReAct prompt from the online hub
prompt = hub.pull('hwchase17/react')  #(name of the repository/the name of the object) this is an object of type PromptTemplate

#print(type(prompt))
#print(prompt.input_variables)
#print(prompt.template)

# Here we are using 3 tools: Python REPL Tool, Wikipedia Tool, and DuckDuckGo Search Tool. The Agent will choose the best tool based on the user's input query.

#1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name= 'Python REPL',
    func= python_repl.run,  #function to run when using this tool
    description='Useful when you need to run Python to answer a question. You should input Python code.'
)



#2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name= 'Wikipedia',
    func= wikipedia.run,  #function to run when using this tool
    description='Useful when you need to look up a topic, country, or person on Wikipedia.'
)

    

#3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name= ' DuckDuckGo Search',
    func= search.run,  #function to run when using this tool
    description='Useful when you need to perform an internet search to find information that another tool can\'t provide.'
)


#Collect all the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# There are basically two main things you have to do:  (1)Load the tools the Agent is going to use  (2)Initialize the Agent using the agent executor and those tools.
agent = create_react_agent(llm, tools,prompt)  # This line of code combines the LLM, The tools and the prompt to create a ReAct Agent.


# set up an agent executor to run the agent. The Executor examines the input and determines the best tool to use based on the user's query. here we are creating the executor to manage the agent's actions and output.
agent_executor = AgentExecutor(
    agent=agent,       #This is the object returned by the create_react_agent() function.
    tools=tools,        #My list of tools
    verbose=True,
    handle_parsing_errors=True, # This argument(handle_parsing_errors) controls how the agent executor handles potential parsing errors that might occur during the execution of tools, or the agent itself.If it is set to True, the executor will attempt to gracefully handle parsing errors by logging the error message for debugging purposes,Continuing with the execution process, potentially skipping the affected tool or part of the agent's response.It will also try to recover and proceed with the remaining steps. If it is set to false which is the default, parsing errors will cause the execution to stop immediately,raising an exception.Using this argument can make agents more robust and less prone to unexpected failures due to parsing issues, specially when working with external tools and unpredictable input.
    max_iterations=10         # This argument(max_iterations) sets a maximum limit on the number of iterations or steps the agent can take during a single execution. Using this argument prevents agents from running indefinitely,which could lead to performance issues,resource exhaustion, or getting stuck in loops.Once the specific number of iteration is reached, the execution will stop even if the agent hasn't fully completed it's task. The optimal value for max_iterations depends on the complexity of the task and the expected execution time.You might need to adjust it based on the specific use case.
)

## Testing the ReAct Agent

#### 1) Test case 1
Asking programming related question to see if the agent uses the python_repl_tool

In [5]:
question = 'Generate the first 20 numbers in the Fibonacci series'

#use the executor to invoke the agent with the question
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
    
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to generate the Fibonacci series in Python to answer this question.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fib_series = [fibonacci(i) for i in range(20)]
fib_series
```I now have the first 20 numbers in the Fibonacci series in Python.
Final Answer: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181

> Finished chain.


In [12]:
# output is a dictionary with two keys input and output.

print(output['input'])


Answer the following questions as best as you can.
Questions: Generate the first 20 numbers in the Fibonacci series



In [13]:
print(output['output'])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]


#### 2) Test case 2
 Asking about a current event question to see if the agent searches the web using DuckDuckGo.

In [6]:
question = 'Who is the current Prime Minister of the UK?'

#use the executor to invoke the agent with the question
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
    
})



> Entering new AgentExecutor chain...
I need to find out who the current Prime Minister of the UK is in Hindi.
Action: DuckDuckGo Search
Action Input: "current Prime Minister of the UK in Hindi"DuckDuckGo Search is not a valid tool, try one of [Python REPL, Wikipedia,  DuckDuckGo Search].I should try using Wikipedia to find the information I need.
Action: Wikipedia
Action Input: Current Prime Minister of the United KingdomPage: Prime Minister of the United Kingdom
Summary: The prime minister of the United Kingdom is the head of government of the United Kingdom. The prime minister advises the sovereign on the exercise of much of the royal prerogative, chairs the Cabinet and selects its ministers. As modern prime ministers hold office by virtue of their ability to command the confidence of the House of Commons, they sit as members of Parliament. The current prime minister is Rishi Sunak of the Conservative Party, who assumed the office on 25 October 2022.
The office of prime minister i

#### 3) Test case 3
Asking question related to history, to see if it chooses Wikipedia tool or not.

In [7]:
question = 'Tell me about Maharana Pratap.'

#use the executor to invoke the agent with the question
output = agent_executor.invoke({
    'input': prompt_template.format(q=question)
    
})



> Entering new AgentExecutor chain...
I should search for information about Maharana Pratap on Wikipedia to answer this question in Hindi.
Action: Wikipedia
Action Input: Maharana PratapPage: Maharana Pratap
Summary: Pratap Singh I (Hindi pronunciation: [Mahārāṇā pratāp] ; c. 9 May 1540 – 19 January 1597), popularly known as Maharana Pratap, was a king of Kingdom of Mewar, a princely state in north-western India in the present-day state of Rajasthan. He is notable for leading the Rajput resistance against the expansionist policy of the Mughal Emperor Akbar including the Battle of Haldighati.

Page: Maharana Pratap Airport
Summary: Maharana Pratap Airport (IATA: UDR, ICAO: VAUD) is a domestic airport serving Udaipur, Rajasthan, India. It is situated at Dabok, located 22 km (14 mi) east of Udaipur. The airport is named after Maharana Pratap who was a Maharana (ruler) of the princely state of Mewar, in north-western India.
The airstrip was used for the first time when a 4-seater Piper S